In [ ]:

// Stan model for a Hidden Markov Model (HMM) with input-driven emission probabilities

data {
    int<lower=1> K;             // Number of hidden states (categories)
    int<lower=1> V;             // Number of possible emissions (observations)
    int<lower=1> T;             // Length of the observation sequence
    int<lower=1, upper=V> w[T]; // Observed emissions at each time step (1-based indexing)
    matrix[T, 2] X;             // Input features influencing emissions (T x 2 matrix)
    vector<lower=0>[K] alpha;   // Dirichlet prior parameters for transition probabilities
}
parameters {
    simplex[K] theta[K];                      // Transition probability matrix (K x K), each row sums to 1
    matrix[3, V] phi_coeffs[K];               // Emission coefficients for each state (K x 3 x V)
                                              // Each state has a matrix of coefficients:
                                              // Rows represent intercept and coefficients for two features
                                              // Columns represent the V possible emissions
}
model {
    // 
    
    Priors for the parameters
    for (k in 1:K) {
        theta[k] ~ dirichlet(alpha);                  // Each row of transition probabilities has a Dirichlet prior
        to_vector(phi_coeffs[k]) ~ normal(0, 1);      // Emission coefficients have a normal prior
    }
    
    // Hidden states (latent variables) and likelihood
    {
        int z[T];                                       // Hidden state sequence (latent states)
        z[1] = categorical_rng(rep_vector(1.0 / K, K)); // Initialize the first state randomly with equal probability
        
        for (t in 1:T) {
            vector[V] linear_pred;                    // Linear predictor for emission probabilities
            
            // Compute the linear predictor for each possible emission
            for (v in 1:V) {
                // For each emission 'v', compute:
                // linear_pred[v] = intercept + feature1_coef * X[t, 1] + feature2_coef * X[t, 2]
                linear_pred[v] = phi_coeffs[z[t], 1, v]               // Intercept term for current state and emission
                                + X[t, 1] * phi_coeffs[z[t], 2, v]    // Coefficient for first input feature
                                + X[t, 2] * phi_coeffs[z[t], 3, v];   // Coefficient for second input feature
            }
            
            // Apply softmax to get emission probabilities that sum to 1
            vector[V] emit_probs = softmax(linear_pred);
            
            // Observe emission 'w[t]' given the emission probabilities
            w[t] ~ categorical(emit_probs);
            
            // State transition for next time step (except for the last time step)
            if (t < T)
                z[t + 1] ~ categorical(theta[z[t]]); // Transition to next state based on current state
        }
    }
}

In [ ]:
model {
    // 
    
    // Priors
    a ~ normal(0, 5);                 // Prior for intercepts
    b ~ normal(0, 5);                 // Prior for slopes
    for (k in 1:K) {
        phi[k] ~ dirichlet(rep_vector(1.0, V)); // Prior for emission probabilities
    }

    // Likelihood
    for (t in 1:T) {
        // Compute linear predictor for each class
        vector[K] eta;
        for (k in 1:K) {
            eta[k] = a[k] + b[k] * u[t]; // Linear predictor for class k
        }
        // Convert to class probabilities using softmax
        vector[K] class_probs = softmax(eta);

        // Mixture likelihood
        real llik = 0;
        for (k in 1:K) {
            llik += class_probs[k] * phi[k][y[t]]; // Probability of y[t] given class k
        }
        target += log(llik); // Increment log-likelihood
    }
}

In [ ]:
// Stan model for a regression mixture model with von Mises emissions
// where latent class probabilities depend on inputs via logistic regression

data {
    int<lower=1> T;                     // Number of observations/time points
    int<lower=1> K;                     // Number of latent classes
    int<lower=1> P;                     // Number of input variables (covariates)
    matrix[T, P] u;                     // Input variables (T x P matrix)
    vector[T] y;                        // Observed outputs (angles in radians)
}

parameters {
    vector[K] a;                        // Intercepts for class probabilities
    matrix[K, P] b;                     // Coefficients for class probabilities (K x P)
    vector[P] w;                        // Coefficients for mean direction (P x 1)
    vector<lower=0>[K] kappa;           // Concentration parameters for von Mises (length K)
}

model {
    //
    
    // Priors
    a ~ normal(0, 5);                   // Prior for intercepts
    to_vector(b) ~ normal(0, 5);        // Prior for slopes
    w ~ normal(0, 5);                   // Prior for mean direction coefficients
    kappa ~ gamma(2, 0.1);              // Prior for concentration parameters

    // Likelihood
    for (t in 1:T) {
        // Compute linear predictor for class probabilities
        vector[K] eta;
        for (k in 1:K) {
            eta[k] = a[k] + b[k] * u[t]'; // Linear predictor for class k
        }
        // Convert to class probabilities using softmax
        vector[K] class_probs = softmax(eta);

        // Compute mean direction for von Mises distribution
        real mu = w' * u[t]'; // Mean direction depends on u_t and w

        // Compute log mixture likelihood using log-sum-exp trick
        vector[K] log_component;
        for (k in 1:K) {
            log_component[k] = log(class_probs[k]) + von_mises_lpdf(y[t] | mu, kappa[k]);
        }
        target += log_sum_exp(log_component); // Increment log-likelihood
    }
}
